In [116]:
import math
import numpy as np
from math import radians, cos, sin, asin, sqrt, atan
from yattag import Doc, indent
from numpy import cross, eye, dot



def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    
    
    bearing = math.atan2(sin(lon2-lon1)*cos(lat2), cos(lat1)*sin(lat2)-sin(lat1)*cos(lat2)*cos(lon2-lon1))
    bearing = math.degrees(bearing)
    bearing = (bearing + 360) % 360
    return c * r , bearing

In [117]:
t1 = [[11.202342416105855,77.35014664877991,0.24136335862807410],[11.20234240119181,77.35014649273029,1.4172300252947410],[11.202342386277765,77.35014633668065,2.5930966919614080],[11.202330003587194,77.35008307659515,0.139037786182009930],[11.202329988673116,77.35008292054553,1.31490445284867690],[11.202329973759035,77.3500827644959,2.49077111951534350],[11.202267635904631,77.35009573018861,0.149236641371927630],[11.202267620990558,77.35009557413902,1.32510330803859460],[11.202267606076484,77.35009541808942,2.50096997470526140],[11.202280048420617,77.35015930236021,0.25156221381799180],[11.202280033506575,77.3501591463106,1.42742888048465870],[11.202280018592536,77.35015899026101,2.60329554715132530],[11.202305026006321,77.35012118948097,0.195300000000000860],[11.20230501109226,77.35012103343136,1.37116666666666780],[11.202304996178201,77.35012087738174,2.54703333333333460]]

In [118]:

n = 5 # 4 + 1 

In [119]:
elementInPath = len(t1)/n

In [120]:
p = list()
temp = list()
flag = 0 
for i in range(len(t1)):
    if i%elementInPath== 0 and i!=0 :
        flag = flag + 1 
        p.append(temp)
        temp = list() 
        
    
    temp.append(t1[i])
    
    if i== len(t1)-1:
        p.append(temp)
    
    
len(p)

5

In [121]:
target = p[:len(p)-1]
ref = p[len(p)-1:][0]


In [122]:
target

[[[11.202342416105855, 77.35014664877991, 0.2413633586280741],
  [11.20234240119181, 77.35014649273029, 1.417230025294741],
  [11.202342386277765, 77.35014633668065, 2.593096691961408]],
 [[11.202330003587194, 77.35008307659515, 0.13903778618200993],
  [11.202329988673116, 77.35008292054553, 1.3149044528486769],
  [11.202329973759035, 77.3500827644959, 2.4907711195153435]],
 [[11.202267635904631, 77.35009573018861, 0.14923664137192763],
  [11.202267620990558, 77.35009557413902, 1.3251033080385946],
  [11.202267606076484, 77.35009541808942, 2.5009699747052614]],
 [[11.202280048420617, 77.35015930236021, 0.2515622138179918],
  [11.202280033506575, 77.3501591463106, 1.4274288804846587],
  [11.202280018592536, 77.35015899026101, 2.6032955471513253]]]

In [123]:

def get_KML(target,ref,name):
    
    total_coord = []
    doc, tag, text = Doc().tagtext()
    
    with tag("kml",('xmlns',"http://www.opengis.net/kml/2.2")):
        with tag("Document",('xmlns',"")):
            with tag("name"):
                text('C2')

            with tag('open'):
                text("1")

            with tag("ExtendedData",('xmlns:mis','www.dji.com')):
                with tag('mis:type'):
                    text('Waypoint')
                with tag('mis:stationType'):     # CHECK THIS SETTING AND WHAT IT IS 
                    text('0')

            with tag('Style',('id','waypointStyle')):
                with tag('IconStyle'):
                    with tag('Icon'):
                        with tag('href'):
                            text('https://cdnen.dji-flighthub.com/static/app/images/point.png')
            with tag('Folder'):
                
                with tag('name'):
                    text('Waypoints')
                with tag('description'):
                    text('Waypoints in the Mission.')
                
                waypointcounter = 1
                trig = 1
                for pathar in target: 
 
                    if trig == 1 :
                        trig= trig*-1
                        ref_new = ref
                        pathar = pathar[::-1]
                        ref_new = ref[::-1]
                    elif trig == -1:
                        trig= trig*-1
                        ref_new = ref 


                    for i in range(len(pathar)):
                        d,b = haversine(  pathar[i][1],pathar[i][0],ref_new[i][1],ref_new[i][0]  )
                        la = pathar[i][0]
                        lo = pathar[i][1]
                        a = pathar[i][2] + 2.5  # ADDING EXTRA ALTITUDE 
#                         print(a)
                        total_coord.append([lo,la,a])
                        
                        with tag('Placemark'):
                            with tag('name'):
                                text(f'Waypoint{waypointcounter}')
                                waypointcounter = waypointcounter + 1 
                            with tag('visibility'):
                                text(f'1')
                            with tag('description'):
                                text(f'Waypoint')
                            with tag('styleUrl'):
                                text(f'#waypointStyle')

                            with tag('ExtendedData',('xmlns:mis', "www.dji.com")):
                                with tag('mis:useWaylineAltitude'):
                                    text('false')

                                with tag('mis:heading'):
                                    if b>180:
                                        yaw = -(360-b)
                                    else: 
                                        yaw = b 
                                    
                                    text(f'{int(yaw)}')
#                                     print(b)

                                with tag('mis:turnMode'):
                                    text('Auto')

                                with tag('mis:gimbalPitch'):
                                    text('0.0')

                                with tag('mis:useWaylineSpeed'):
                                    text('false')

                                with tag('mis:speed'):
                                    text('1.0')

                                with tag('mis:useWaylineHeadingMode'):
                                    text('true')

                                with tag('mis:useWaylinePointType'):
                                    text('true')

                                with tag('mis:pointType'):
                                    text('LineStop')

                                with tag('mis:cornerRadius'):
                                    text('0.2')

                                with tag('mis:actions',('param', "2000"),('accuracy', "0"),('cameraIndex', "0"),('payloadType', "0")
                                            ,('payloadIndex', "0")):
                                    text('Hovering')
                                    
                                with tag('mis:actions',('param', "0"),('accuracy', "0"),('cameraIndex', "0"),('payloadType', "0")
                                            ,('payloadIndex', "0")):
                                    text('ShootPhoto')
                                    
                            with tag("Point"):
                                with tag('altitudeMode'):
                                        text('relativeToGround')

                                with tag('coordinates'):
                                        text(f'{lo},{la},{a}')

            with tag('Placemark'):
                with tag('name'):
                    text(f'Wayline')

                with tag('description'):
                    text(f'Wayline')

                with tag('visibility'):
                    text(f'1')

                with tag('ExtendedData',('xmlns:mis', "www.dji.com")):

                    with tag('mis:altitude'):
                        text('20.0')

                    with tag('mis:autoFlightSpeed'):
                        text(f'1.0')

                    with tag('mis:actionOnFinish'):
                        text('Hover')

                    with tag('mis:headingMode'):
                        text('UsePointSetting')

                    with tag('mis:gimbalPitchMode'):
                        text('UsePointSetting')

                    with tag('mis:powerSaveMode'):
                        text('false')

                    with tag('mis:waypointType'):
                        text('LineStop')

                    with tag('mis:droneInfo'):
                        with tag('mis:droneType'):
                            text('COMMON')
                        with tag('mis:advanceSettings'):
                            text('false')
                        with tag('mis:droneCameras'):
                            text('')
                        with tag('mis:droneHeight'):
                            with tag('mis:useAbsolute'):
                                text('false')
                            with tag('mis:hasTakeoffHeight'):
                                text('false')
                            with tag('mis:takeoffHeight'):
                                text('0.0')


                with tag('styleUrl'):
                    text('#waylineGreenPoly')

                with tag("LineString"):
                    with tag('tessellate'):
                            text('1')

                    with tag('altitudeMode'):
                            text('relativeToGround')

                    with tag('coordinates'):
                        coordinates_total = ''

                        for i in total_coord:
                            coordinates_total = coordinates_total + str(i[0]) + ',' + str(i[1]) + ',' + str(i[2])[:5] + ' '

                        text(f'{coordinates_total}')


    result = indent(
        doc.getvalue(),
        indentation = ' '*4,
        newline = '\r\n'
    )

    

    save_path_file = name
    with open(save_path_file, "w") as f: 
        f.write(result)  




In [124]:
get_KML(target,ref,'091620-tree-05.kml')

In [125]:
haversine(77.34990317,11.2020383,77.34984849,11.20212160)

(0.011016690899817088, 327.22185293188954)

In [126]:
18

18

In [127]:
!open .
